<a href="https://colab.research.google.com/github/davidenko2000/ProjectR/blob/main/Resnet18_cifar128NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1 

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
       

        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
       
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
       

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [2]:
#Training function
def train(epoch):
    print('\nEpoch: %d' % epoch)
    train_loss = 0
    correct = 0
    total = 0
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))   
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Train -> Loss: %.3f | Acc: %.3f%%'
                  % (train_loss/(len(trainloader)), 100.*correct/total))
#Test function
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print('Test -> Loss: %.3f | Acc: %.3f%%'
                  % (test_loss/(len(testloader)), 100.*correct/total))

In [3]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

best_acc = 0  
start_epoch = 0  

#Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

#hyperparameters
classes = ('plane', 'car', 'bird', 'cat', 'deer',
         'dog', 'frog', 'horse', 'ship', 'truck')

#torch.save(net, 'resnet_cifar.pth')
#Model
print('==> Building model..')

net = ResNet18()
net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-1,
                      momentum=0.9, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
start_time = time.time()

for epoch in range(start_epoch, start_epoch+30):
      train(epoch)
      test(epoch)
      scheduler.step()



==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
==> Building model..

Epoch: 0
Time elapsed: 0.00 min
Train -> Loss: 1.883 | Acc: 32.114%
Test -> Loss: 1.570 | Acc: 41.530%

Epoch: 1
Time elapsed: 3.19 min
Train -> Loss: 1.636 | Acc: 39.024%
Test -> Loss: 1.422 | Acc: 46.920%

Epoch: 2
Time elapsed: 6.38 min
Train -> Loss: 1.379 | Acc: 49.254%
Test -> Loss: 1.218 | Acc: 55.210%

Epoch: 3
Time elapsed: 9.56 min
Train -> Loss: 1.211 | Acc: 55.740%
Test -> Loss: 1.081 | Acc: 60.050%

Epoch: 4
Time elapsed: 12.74 min
Train -> Loss: 1.100 | Acc: 60.402%
Test -> Loss: 1.146 | Acc: 59.530%

Epoch: 5
Time elapsed: 15.92 min
Train -> Loss: 1.003 | Acc: 64.102%
Test -> Loss: 0.978 | Acc: 65.090%

Epoch: 6
Time elapsed: 19.10 min
Train -> Loss: 0.924 | Acc: 67.184%
Test -> Loss: 0.923 | Acc: 67.100%

Epoch: 7
Time elapsed: 22.28 min
Train -> Loss: 0.855 | Acc: 69.780%
Test -> Loss: 0.843 | Acc: 70.380%

Epoch: 8
Time elapsed: 25.46 min
Train -> Loss: 0.806